In [ ]:
# install all requirements
!pip3 install -r ../requirements.txt

In [118]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats



import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import sddk

In [119]:
import textnet

In [135]:
reload(textnet)

<module 'textnet' from '/home/jovyan/scripts/textnet.py'>

In [116]:
resp_text = requests.get("https://raw.githubusercontent.com/kasev/textnet/master/textnet/__init__.py").text
exec(resp_text)

In [5]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [6]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201119.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,n_sentences,lemmatized_sentences,tlg_epithet
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,3252,"[[ἄρχω, Φοῖβος, παλαιγενεών, κλέος, φώς, μιμνή...",Epici/-ae
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,6068,"[[θουκυδιδής, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",Historici/-ae
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,10245,"[[φιλοσοφία, ἔργον, ἔνειμι, φάσις, βάρβαρος, ἄ...",Biographi
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,1982,"[[ψιθύρισμα, πίτυς, αἰπολέ, τῆνος, παγαισί, με...",Bucolici
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,152,"[[ῥόδον, δροσοεντά, καταπυκνός, ἐκεινά, ἕρπυλλ...",Bucolici


In [7]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["n_lemmata"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["n_lemmata"].sum() # previously we had 13925726, then 13713183

16184099

In [8]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [9]:
subcorpora_dict = {
    "Pagan (8th c. BCE - 4th c. CE)" : AGT[(AGT["provenience"]=="pagan") & (AGT["date_avr"] < 4)],
    "Pagan (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")]}

In [10]:
subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "doccount" : doccount, "n_sentences": sents_len,"sentences" : sents})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,doccount,n_sentences,sentences
0,Pagan (8th c. BCE - 4th c. CE),12965219,774,1039409,"[[ἄρχω, Φοῖβος, παλαιγενεών, κλέος, φώς, μιμνή..."
1,Pagan (1st - 4th c. CE),8692448,411,723956,"[[γεωγραφία, σοσσιέ, σενεκιών, ἱστορικοί, διαφ..."
2,Septuagint,799083,55,57856,"[[ἔν, ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ἀό..."
3,Philo of Alexandria,643011,31,82966,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,Titus Flavius Josephus,469580,4,19707,"[[ἱστορία, συγγραφείν, βούλομαι, εἷς, αὐτός, ὁ..."
5,Paul of Tarsus,24066,7,1632,"[[δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπόστολος, ..."
6,New Testament,137787,27,8698,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
7,Christian (1st - 4th c. CE),2975644,131,291084,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."


# term2term & term2doc matraces with sklearn - Paul as example

In [11]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [12]:
len(docs)

1632

In [13]:
print(docs[:3])

[['δοῦλος', 'Ἰησοῦς', 'Χριστός', 'κλητός', 'ἀπόστολος', 'ἀφωρισμενός', 'εὐαγγέλιον', 'θεός', 'προεπηγγειλατό', 'προφήτης', 'γραφή', 'ἁγιαίς', 'υἱός', 'γίγνομαι', 'σπέρμα', 'Δαυίδ', 'σάρξ', 'ὁρισθεντός', 'υἱός', 'θεός', 'δύναμις', 'πνεῦμα', 'ἁγιωσυνής', 'ἀνάστασις', 'νεκρός', 'Ἰησοῦς', 'Χριστός', 'κύριος', 'λαμβάνω', 'χάρις', 'ἀποστολή', 'ὑπακοή', 'πίστις', 'ἔθνος', 'ὄνομα', 'κλητός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'οὐσίν', 'Ῥώμη', 'ἀγαπητός', 'θεός', 'κλητός', 'ἅγιος'], ['χάρις', 'εἰρήνη', 'θεός', 'πατήρ', 'κύριος'], ['πρῶτος', 'εὐχαριστέω', 'θεός', 'Ἰησοῦς', 'Χριστός', 'πᾶς', 'πίστις', 'καταγγέλλω', 'ὅλος', 'κοσμῴ.']]


In [14]:
vocabulary =  list(set([word for sent in docs for word in sent]))
bow = CountVectorizer(vocabulary=vocabulary)
bow_term2doc = bow.fit_transform([" ".join(doc) for doc in docs]) ### run the model
term2term_bow = (bow_term2doc.T * bow_term2doc)

In [15]:
term2term_bow

<2161x2161 sparse matrix of type '<class 'numpy.int64'>'
	with 62622 stored elements in Compressed Sparse Column format>

In [107]:
G = nx.from_numpy_matrix(term2term_bow.todense()) # from_pandas_adjacency()
vocab_dict = dict(zip(range(len(vocabulary)), vocabulary))
G = nx.relabel_nodes(G, vocab_dict)
print(G.number_of_nodes(), G.number_of_edges())

2161 32300


In [108]:
list(G.edges.data())[:5]

[('χριστός', 'χριστός', {'weight': 312}),
 ('χριστός', 'δεῖ', {'weight': 1}),
 ('χριστός', 'καταντάω', {'weight': 3}),
 ('χριστός', 'τολμάω', {'weight': 2}),
 ('χριστός', 'ποιέω', {'weight': 5})]

In [69]:
# remove 
edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [70]:
total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
total_weight

49906

In [83]:
weights = sorted([int(n) for n in nx.get_edge_attributes(G, "weight").values()], reverse=True)

In [102]:
weight_threshold = 0.1
index_position = int(len(weights) * weight_threshold)
minimal_weight_value = weights[index_position]

In [103]:
edges_to_remove = []
for edge in G.edges:
    if G[edge[0]][edge[1]]["weight"] < minimal_weight_value:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [104]:
len(G.nodes)

2161

In [105]:
len(G.edges)

3317

In [106]:
list(G.edges.data())[:5]

[('χριστός',
  'καταντάω',
  {'weight': 3,
   'norm_weight': 6e-05,
   'distance': 0.33333,
   'norm_distance': 16666.66667}),
 ('χριστός',
  'ποιέω',
  {'weight': 5,
   'norm_weight': 0.0001,
   'distance': 0.2,
   'norm_distance': 10000.0}),
 ('χριστός',
  'οὐσίν',
  {'weight': 3,
   'norm_weight': 6e-05,
   'distance': 0.33333,
   'norm_distance': 16666.66667}),
 ('χριστός',
  'ἀγάπη',
  {'weight': 17,
   'norm_weight': 0.00034,
   'distance': 0.05882,
   'norm_distance': 2941.17647}),
 ('χριστός',
  'βεβαιόω',
  {'weight': 6,
   'norm_weight': 0.00012,
   'distance': 0.16667,
   'norm_distance': 8333.33333})]

In [121]:
edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [122]:
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)

In [131]:
list(G.edges.data())[:5]

[('χριστός',
  'δεῖ',
  {'weight': 1,
   'norm_weight': 2e-05,
   'distance': 1.0,
   'norm_distance': 50000.0}),
 ('χριστός',
  'καταντάω',
  {'weight': 3,
   'norm_weight': 6e-05,
   'distance': 0.33333,
   'norm_distance': 16666.66667}),
 ('χριστός',
  'τολμάω',
  {'weight': 2,
   'norm_weight': 4e-05,
   'distance': 0.5,
   'norm_distance': 25000.0}),
 ('χριστός',
  'ποιέω',
  {'weight': 5,
   'norm_weight': 0.0001,
   'distance': 0.2,
   'norm_distance': 10000.0}),
 ('χριστός',
  'χρίω',
  {'weight': 1,
   'norm_weight': 2e-05,
   'distance': 1.0,
   'norm_distance': 50000.0})]

In [145]:
network = G.copy(as_view=False)

In [154]:
def construct_ego_network(source_network, term, num_of_neighbours):
    length, path = nx.single_source_dijkstra(source_network, term, target=None, weight="distance")
    shortest_nodes = list(length.keys())[0:num_of_neighbours+1]
    path_values_sorted = [dict_pair[1] for dict_pair in sorted(path.items(), key=lambda pair: list(length.keys()).index(pair[0]))]
    path_edges = []
    for path_to_term in path_values_sorted[1:num_of_neighbours+1]:
        path_edges.extend([tuple(sorted(bigram)) for bigram in nltk.bigrams(path_to_term)])
    shortest_edges = list(set(path_edges))
    ego_network = source_network.copy(as_view=False)
    nodes_to_remove = []
    for node in ego_network.nodes:
        if node not in shortest_nodes:
            nodes_to_remove.append(node)
    for element in nodes_to_remove:
        ego_network.remove_node(element)    
    edges_to_remove = []
    for edge in ego_network.edges:
        if edge not in shortest_edges:
            if (edge[1],edge[0]) not in shortest_edges:
                edges_to_remove.append(edge)
    for element in edges_to_remove:
        ego_network.remove_edge(element[0], element[1])
    return ego_network

In [156]:
ego_network = construct_ego_network(network, "χριστός", 30)
ego_network.edges.data()

EdgeDataView([('χριστός', 'νεκρός', {'weight': 25, 'norm_weight': 0.0005, 'distance': 0.04, 'norm_distance': 2000.0}), ('χριστός', 'κύριος', {'weight': 53, 'norm_weight': 0.00106, 'distance': 0.01887, 'norm_distance': 943.39623}), ('χριστός', 'εὐαγγέλιον', {'weight': 25, 'norm_weight': 0.0005, 'distance': 0.04, 'norm_distance': 2000.0}), ('χριστός', 'θεός', {'weight': 136, 'norm_weight': 0.00273, 'distance': 0.00735, 'norm_distance': 366.30037}), ('χριστός', 'πᾶς', {'weight': 69, 'norm_weight': 0.00138, 'distance': 0.01449, 'norm_distance': 724.63768}), ('χριστός', 'ἐγείρω', {'weight': 23, 'norm_weight': 0.00046, 'distance': 0.04348, 'norm_distance': 2173.91304}), ('χριστός', 'πίστις', {'weight': 37, 'norm_weight': 0.00074, 'distance': 0.02703, 'norm_distance': 1351.35135}), ('χριστός', 'ἔχω', {'weight': 26, 'norm_weight': 0.00052, 'distance': 0.03846, 'norm_distance': 1923.07692}), ('ἔθνος', 'θεός', {'weight': 26, 'norm_weight': 0.00052, 'distance': 0.03846, 'norm_distance': 1923.0769

In [157]:
def extract_ego_network_data(ego_network):
    ego_network_data_prec = sorted(list(ego_network.edges.data("weight")), key=lambda tup: int(tup[2]), reverse=True)
    ego_network_data_complete = []
    for tup in ego_network_data_prec:
      if tup[1] == term:
        ego_network_data_complete.append([tup[1], tup[0], int(tup[2]), round(1 / int(tup[2]), 5)])
      else:
        ego_network_data_complete.append([tup[0], tup[1], int(tup[2]), round(1 / int(tup[2]), 5)])
    return ego_network_data_complete

In [158]:
extract_ego_network_data(ego_network)

[['χριστός', 'θεός', 136, 0.00735],
 ['θεός', 'εἰμί', 91, 0.01099],
 ['χριστός', 'πᾶς', 69, 0.01449],
 ['χριστός', 'κύριος', 53, 0.01887],
 ['θεός', 'πνεῦμα', 50, 0.02],
 ['χάρις', 'θεός', 48, 0.02083],
 ['θεός', 'οἶδα', 47, 0.02128],
 ['θεός', 'γίγνομαι', 46, 0.02174],
 ['θεός', 'ἄνθρωπος', 39, 0.02564],
 ['χριστός', 'πίστις', 37, 0.02703],
 ['θεός', 'πατήρ', 36, 0.02778],
 ['θεός', 'ἄλλʼ', 34, 0.02941],
 ['λόγος', 'θεός', 33, 0.0303],
 ['θεός', 'πολύς', 33, 0.0303],
 ['θεός', 'νόμος', 33, 0.0303],
 ['σάρξ', 'θεός', 30, 0.03333],
 ['θεός', 'ἅγιος', 30, 0.03333],
 ['θεός', 'δόξα', 30, 0.03333],
 ['θεός', 'ἀδελφός', 29, 0.03448],
 ['θεός', 'δίδωμι', 28, 0.03571],
 ['θεός', 'αὐτός', 27, 0.03704],
 ['θεός', 'κόσμος', 27, 0.03704],
 ['χριστός', 'ἔχω', 26, 0.03846],
 ['ἔθνος', 'θεός', 26, 0.03846],
 ['θεός', 'δικαιοσύνη', 26, 0.03846],
 ['χριστός', 'νεκρός', 25, 0.04],
 ['χριστός', 'εὐαγγέλιον', 25, 0.04],
 ['θεός', 'υἱός', 25, 0.04],
 ['θεός', 'δύναμις', 25, 0.04],
 ['χριστός', 'ἐγείρω', 2

In [160]:
reload(textnet)

<module 'textnet' from '/home/jovyan/scripts/textnet.py'>

In [185]:
networkx_object = ego_network.copy()

In [186]:
pos_2d=nx.kamada_kawai_layout(networkx_object, weight="weight_norm")
pos_2d

{'χριστός': array([-0.50099769,  0.11564409]),
 'νεκρός': array([-1.        , -0.02253037]),
 'ἔθνος': array([0.48812218, 0.19575123]),
 'χάρις': array([0.60659051, 0.41026091]),
 'σάρξ': array([0.72929812, 0.23023622]),
 'κύριος': array([-0.53206425,  0.75419602]),
 'εὐαγγέλιον': array([-0.98786204,  0.22802799]),
 'λόγος': array([0.21016303, 0.33031328]),
 'θεός': array([ 0.18755395, -0.04330021]),
 'πατήρ': array([0.44035978, 0.51532323]),
 'δικαιοσύνη': array([0.25366513, 0.57865158]),
 'υἱός': array([0.05075191, 0.53201987]),
 'ἄλλʼ': array([-0.08307975,  0.37193937]),
 'πνεῦμα': array([-0.13144264,  0.16073574]),
 'πᾶς': array([-0.72740916,  0.63469799]),
 'αὐτός': array([-0.18862126, -0.08699733]),
 'κόσμος': array([-0.23769146, -0.29798555]),
 'γίγνομαι': array([ 0.04400793, -0.38910001]),
 'εἰμί': array([-0.18753142, -0.50054991]),
 'ἐγείρω': array([-0.88895519,  0.45855117]),
 'πίστις': array([-0.93199316, -0.25167307]),
 'ἔχω': array([-0.80879593, -0.44468618]),
 'δύναμις': 

In [187]:
# normalize positions
dmin=1
ncenter=0
for key in pos_2d:
    x,y=pos_2d[key]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=key
        dmin=d

In [188]:
p =nx.single_source_shortest_path_length(networkx_object, ncenter)

In [189]:
p

{'πατήρ': 0,
 'θεός': 1,
 'χριστός': 2,
 'δικαιοσύνη': 2,
 'νόμος': 2,
 'οἶδα': 2,
 'κόσμος': 2,
 'σάρξ': 2,
 'αὐτός': 2,
 'δόξα': 2,
 'γίγνομαι': 2,
 'ἅγιος': 2,
 'πολύς': 2,
 'ἀδελφός': 2,
 'δίδωμι': 2,
 'ἔθνος': 2,
 'λόγος': 2,
 'υἱός': 2,
 'ἄλλʼ': 2,
 'πνεῦμα': 2,
 'εἰμί': 2,
 'δύναμις': 2,
 'χάρις': 2,
 'ἄνθρωπος': 2,
 'νεκρός': 3,
 'ἐγείρω': 3,
 'κύριος': 3,
 'πίστις': 3,
 'ἔχω': 3,
 'πᾶς': 3,
 'εὐαγγέλιον': 3}

In [ ]:
def draw_2d_network(networkx_object, file_name, mode):
    '''take networkX object and draw it'''
    pos_2d=nx.kamada_kawai_layout(networkx_object, weight="weight_norm")
    nx.set_node_attributes(networkx_object, pos_2d, "pos_2d")
    dmin=1
    ncenter=0
    Edges = list(networkx_object.edges)
    L=len(Edges)
    labels= list(networkx_object.nodes)
    N = len(labels)
    distance_list = [float(distance[2]) for distance in list(networkx_object.edges.data("distance"))]
    weight_list = [int(float(weight[2])) for weight in list(networkx_object.edges.data("weight"))]
    for n in pos_2d:
        x,y=pos_2d[n]
        d=(x-0.5)**2+(y-0.5)**2
        if d<dmin:
            ncenter=n
            dmin=d
    p =nx.single_source_shortest_path_length(networkx_object, ncenter)
    adjc= [len(one_adjc) for one_adjc in list((nx.generate_adjlist(networkx_object)))]
    middle_node_trace = go.Scatter(
        x=[],
        y=[],
        opacity=0,
        text=weight_list,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            opacity=0
            )
        )
    for Edge in Edges:
        x0,y0 = networkx_object.nodes[Edge[0]]["pos_2d"]
        x1,y1 = networkx_object.nodes[Edge[1]]["pos_2d"]
        middle_node_trace['x'] += tuple([(x0+x1)/2])
        middle_node_trace['y'] += tuple([(y0+y1)/2])
    edge_trace1 = go.Scatter(
        x=[], y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=1,color="#000000"),
        )
    edge_trace2 = go.Scatter(
        x=[],y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=0.7,color="#404040"),
        )
    edge_trace3 = go.Scatter(
        x=[], y=[],
        #hoverinfo='none',
        mode='lines',
        line=dict(width=0.5,color="#C0C0C0"),
        )
    best_5percent_norm_weight = sorted(list(networkx_object.edges.data("norm_weight")), key=lambda x: x[2], reverse=True)[int((len(networkx_object.edges.data("norm_weight")) / 100) * 5)][2]
    best_20percent_norm_weight = sorted(list(networkx_object.edges.data("norm_weight")), key=lambda x: x[2], reverse=True)[int((len(networkx_object.edges.data("norm_weight")) / 100) * 20)][2]
    for edge in networkx_object.edges.data():
        if edge[2]["norm_weight"] >= best_5percent_norm_weight:
            x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
            x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
            edge_trace1['x'] += tuple([x0, x1, None])
            edge_trace1['y'] += tuple([y0, y1, None])
        else:
            if edge[2]["norm_weight"] >= best_20percent_norm_weight:
                x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
                x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
                edge_trace2['x'] += tuple([x0, x1, None])
                edge_trace2['y'] += tuple([y0, y1, None])
            else:
                x0, y0 = networkx_object.nodes[edge[0]]['pos_2d']
                x1, y1 = networkx_object.nodes[edge[1]]['pos_2d']
                edge_trace3['x'] += tuple([x0, x1, None])
                edge_trace3['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        #name=[],
        text=[],
        textposition='bottom center',
        mode='markers+text',
        hovertext=adjc,
        hoverinfo='text',
        marker=dict(
            ###showscale=True,
            showscale=False, ### change to see scale
            colorscale='Greys',
            reversescale=True,
            color=[],
            size=7,
            colorbar=dict(
                thickness=15,
                title='degree',
                xanchor='left',
                titleside='right'
                ),
            line=dict(width=1)
            )
        )

    for node in networkx_object.nodes():
        x, y = networkx_object.nodes[node]['pos_2d']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace["text"] += tuple([node])
        ### original version: node_trace["text"] += tuple([node])

    ### Color Node Points
    for node, adjacencies in enumerate(nx.generate_adjlist(networkx_object)):
        node_trace['marker']['color'] += tuple([len(adjacencies)])
        ###node_info = ' of connections: '+str(len(adjacencies))
        ###node_trace['something'].append(node_info)

    fig = go.Figure(data=[edge_trace1, edge_trace2, edge_trace3, node_trace, middle_node_trace],
        layout=go.Layout(
            plot_bgcolor='rgba(0,0,0,0)',
            autosize=False,
            width=500,
            height=500,
            #title=file_name,
            titlefont=dict(size=16),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=10,l=10,r=10, t=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
            ))
    return fig

In [171]:
#textnet.configure_plotly_browser_state()
fig = textnet.draw_2d_network(ego_network, "G.png", "offline")

In [172]:
type(fig)

plotly.graph_objs._figure.Figure

In [173]:
textnet.configure_plotly_browser_state()
fig

In [126]:
textnet.configure_plotly_browser_state()
fig = textnet.draw_2d_network(G, "G.png", "offline")

KeyboardInterrupt: 

In [170]:
import plotly.graph_objects as go



In [174]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
#fig.show()

In [175]:
type(fig)

plotly.graph_objs._figure.Figure

In [ ]:
bow = CountVectorizer(vocabulary=vocabulary)
bow_term2doc = bow.fit_transform([" ".join(doc) for doc in docs]) ### run the model

term2term_bow = (bow_term2doc.T * bow_term2doc)


tfidf =  TfidfVectorizer(vocabulary=vocabulary)
tfidf_term2doc = tfidf.fit_transform([" ".join(doc) for doc in docs]) ### run the model

svd = TruncatedSVD(n_components=150, n_iter=5, random_state=42)
svd_bow = svd.fit(bow_term2doc)

svd = TruncatedSVD(n_components=150, n_iter=5, random_state=42)
svd_tfidf = svd.fit(tfidf_term2doc)
    #term2doc = pd.DataFrame(Xc.toarray(), columns=vec.get_feature_names(), index=vec.get_feature_names())
lsa_bow = pd.DataFrame(svd_bow.components_, columns=vocabulary)
lsa_tfidf = pd.DataFrame(svd_tfidf.components_, columns=vocabulary)

In [ ]:
words = ["δίκαιος", "δίκη", "δικαιοσύνη"]

In [ ]:
term2term_bow_df = pd.DataFrame(term2term_bow.todense(), columns=vocabulary, index=vocabulary)
term2term_bow_df

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
πρᾶγμα,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ἡσσάομαι,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
θλίβω,0,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
θλίψιν,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
διδαχῆς,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ὑπακοην,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,4,0,0,0,0
ἀντικειμένωνἥτις,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,1,0,0
ἔνδειξις,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,6,0,0
ὑποτάσσω,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12,0


In [ ]:
sorted_df = term2term_bow_df.sort_values("δικαιοσύνη", ascending=False)[["δικαιοσύνη"]]
print(sorted_df.head(20))

            δικαιοσύνη
δικαιοσύνη          62
πίστις              25
θεάομαι             18
νόμος               16
λογίζομαι           11
χριστοῦ             10
ἁμαρτία             10
ἰησοῦ               10
θεός                 9
πᾶς                  9
πιστεύω              7
περιτομή             7
ἀκροβυστία           6
πατήρ                6
ἀβραάμ               6
ἀνομία               5
βασιλεύω             5
δοῦλος               5
κύριος               5
ἁμάρτημα             5


In [ ]:
sorted_df = term2term_bow_df.sort_values("δίκαιος", ascending=False)[["δίκαιος"]]
print(sorted_df.head(20))

            δίκαιος
δίκαιος          14
πίστις            8
ὅσος              6
θεός              6
νόμος             5
πᾶς               4
θεάομαι           4
ἕνος              3
δικαιοσύνη        3
ἁμός              3
ζάω               3
δικαιόω           2
ἀλλ               2
ποιέω             2
πολύς             2
ἅγιος             2
καθίστημι         2
ἔχω               2
γράφω             2
εἶμι              2


In [ ]:
lsa_model_data

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
0,0.003708,0.000052,0.018253,0.000661,0.000759,0.000499,0.000129,0.001352,0.000011,0.001212,...,-6.639306e-16,0.006197,0.000482,0.000551,0.067805,0.012405,0.001671,0.005543,0.014762,0.001498
1,-0.001711,-0.000088,0.019553,-0.001290,-0.000133,-0.000006,-0.000068,-0.001622,0.000023,0.000156,...,-8.387424e-15,-0.003687,0.000134,-0.000948,-0.018718,-0.002153,0.000014,0.001900,-0.017855,-0.003475
2,-0.001938,0.000137,-0.002422,0.002927,0.000159,0.000678,0.000282,-0.000731,0.000011,0.001485,...,2.157429e-14,0.002290,0.000561,0.002798,0.067655,0.000908,0.002165,0.003588,-0.019197,-0.003370
3,-0.000222,-0.000004,-0.019747,-0.001763,0.000364,0.001681,-0.000111,0.000080,-0.000030,0.004362,...,-1.263571e-13,-0.012455,0.000969,-0.001761,-0.010202,0.017871,0.004209,0.011782,0.010564,-0.000816
4,-0.000502,-0.000153,0.002632,-0.000557,0.003583,0.000096,-0.000091,-0.001782,-0.000024,0.000647,...,3.878321e-13,0.000985,-0.000747,-0.000634,-0.038459,-0.009959,-0.001097,0.004457,-0.003471,-0.000706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,-0.010291,0.003474,-0.049004,-0.001600,-0.011729,0.012179,-0.004205,-0.002103,0.003453,-0.016910,...,-8.785520e-07,0.014278,-0.002597,-0.005485,-0.006336,0.026199,-0.013394,-0.005216,0.001714,-0.009270
146,-0.028570,0.007097,0.005474,-0.006091,0.003053,0.004164,0.001158,0.001073,-0.001872,-0.020473,...,-7.591400e-08,-0.000330,0.004361,0.010873,-0.017895,0.000099,-0.006938,-0.027157,0.002984,-0.024311
147,-0.005424,-0.004964,0.012694,-0.006861,-0.005116,0.001409,0.001862,0.009463,0.000358,0.006379,...,-5.241422e-07,0.028197,0.001656,-0.004227,-0.005124,-0.052964,0.022026,0.048630,0.046711,0.013198
148,0.021749,-0.000103,0.003712,-0.005903,-0.006872,-0.016247,0.007232,0.009815,-0.003137,-0.054888,...,-5.550533e-07,0.046293,0.007101,0.002207,-0.021809,-0.019193,-0.008599,0.001844,-0.025452,-0.006316


In [ ]:
doc2term_df = pd.DataFrame(bow_doc2term.toarray(), columns=vocabulary)
doc2term_df.head()

,πρᾶγμα,ἡσσάομαι,θλίβω,θλίψιν,διδαχῆς,ψάλλω,ἐλάχιστόν,διατάσσω,ποῦ,θερίζω,...,εἴδωλόν,φοβέω,θαυμάζω,ὀδυρμός,εὐαγγέλιον,ὑπακοην,ἀντικειμένωνἥτις,ἔνδειξις,ὑποτάσσω,συγχαίρω
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def lsa_model(docs):
    vocabulary =  list(set([word for doc in docs for word in doc]))
    tfidf =  TfidfVectorizer(vocabulary=vocabulary) ### initiaze the model
    X = tfidf.fit_transform([" ".join(doc) for doc in docs]) ### run the model
    Xc = (X.T * X)
    svd = TruncatedSVD(n_components=50, n_iter=5, random_state=42)
    svd.fit(Xc)
    #term2doc = pd.DataFrame(Xc.toarray(), columns=vec.get_feature_names(), index=vec.get_feature_names())
    lsa_model_data = pd.DataFrame(svd.components_, columns=vec.get_feature_names())
    return lsa_model_data, Xc#term2doc

In [ ]:
model, term2doc = lsa_model(christian_docs)

In [ ]:
len(model.columns)

117515

In [ ]:
def get_most_similar(model_df, target_term, number):
  all_similar = []
  for term in model_df.columns:
    similarity = (term, cosine_similarity([model_df[target_term],  model_df[term]])[0][1])
    all_similar.append(similarity)
  return sorted(all_similar, key=lambda number: number[1], reverse=True)[1:number]

In [ ]:
words = ["δίκαιος", "δίκη", "δικαιοσύνη"]

In [ ]:
neighbours = {}  
for word in words:
    neighbours[word] = get_most_similar(model, word, 10)

In [ ]:
pd.DataFrame(neighbours)

,δίκαιος,δίκη,δικαιοσύνη
0,"(ιειμένην, 0.9834371832818561)","(μετελεύσονται, 0.9384522490347709)","(καθυπηρετεῖ, 0.9710013738851778)"
1,"(εὐστοχίου, 0.9830720889621722)","(ἐπικουροί, 0.9384522490347709)","(ἐνδύσονται, 0.9706819000158255)"
2,"(λογιωτάτου, 0.9830720889621722)","(ἐριννύες, 0.9384522490347709)","(σιοί, 0.9706819000158255)"
3,"(λελύπηται, 0.9830720889621722)","(καταπεπηγμένων, 0.9344969432505092)","(περικτᾶται, 0.9699396541878471)"
4,"(τηρούσης, 0.9827599613491105)","(ἐξέτισας, 0.9182233983592916)","(ἡδυπαθείᾳ, 0.9699396541878471)"
5,"(μιμείσθω, 0.9826270670854331)","(γλωσσαργία, 0.8999555833389017)","(ἅνευ, 0.9695269024725653)"
6,"(ἰὼδ, 0.9822301096106603)","(κρυσταλλοειδεῖ, 0.8997426356399815)","(χρωννύντες, 0.9695269024725653)"
7,"(ἐπαράντων, 0.9820947343036778)","(διώκτας, 0.8970726088109655)","(πρυπορεύσεται, 0.9662484878532644)"
8,"(πειλθαρχεῖν, 0.9820947343036778)","(προςώποις, 0.8861867911971005)","(ἐπιφαινέσθω, 0.9660042345965665)"
